# Modules

In [37]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime, date, time
import seaborn as sns
from scipy import stats
import scipy.stats as st
import glob

# Dataset

The dataset used in this analysis consists of beer reviews from two beer rating websites,**BeerAdvocate** and **RateBeer**, for a period ranging from 2001 to 2017. For each website, we have 5 files:
- users.csv: metadata about reviewers
- beers.csv : metadata about reviewed beers
- breweries.csv : metadata about breweries
- ratings.txt : all reviews given by users, including numerical ratings and sometimes textual reviews
- reviews.txt : only reviews given by users that include both numerical ratings and textual reviews

In our analysis, we will not use textual reviews. Thus, we will only use ratings.txt files and not reviews.txt files, as we will use all reviews, whether or not they include textual reviews.

### Load data into Dataframes

The .csv files are not too large and can efficiently be loaded into DataFrames.

In [38]:
BA_DATA_FOLDER = 'data/BeerAdvocate/'
RB_DATA_FOLDER = 'data/RateBeer/'

BA_USERS = BA_DATA_FOLDER+"users.csv"
BA_BEERS = BA_DATA_FOLDER+"beers.csv"
BA_BREWERIES = BA_DATA_FOLDER+"breweries.csv"

RB_USERS = RB_DATA_FOLDER+"users.csv"
RB_BEERS = RB_DATA_FOLDER+"beers.csv"
RB_BREWERIES = RB_DATA_FOLDER+"breweries.csv"

In [39]:
ba_users = pd.read_csv(BA_USERS)
ba_beers = pd.read_csv(BA_BEERS)
ba_breweries = pd.read_csv(BA_BREWERIES)

rb_users = pd.read_csv(RB_USERS)
rb_beers = pd.read_csv(RB_BEERS)
rb_breweries = pd.read_csv(RB_BREWERIES)

On the other hand, the ratings.txt files are extremely large, and trying to load them directly into DataFrames leads to kernel freezes. In order to circumvent this problem, we wrote a script (review_parser.py, located in src/scripts), which processes each rating file by dividing it into parts, parsing each part, and saving as JSON. In the notebook, we then load the different JSON files into DataFrames, that we concatenate. Dividing the large .txt files into smaller JSON chunks and then loading each chunk separately, avoids trying to load the entire file into memory at once, which can cause kernel freezes due to memory overload. In addition, JSON is a format that pandas can read efficiently.

In [40]:
# Load BeerAdvocate ratings stored in json files into a single DataFrame
ba_json_files = glob.glob(BA_DATA_FOLDER+'*.json')
ba_df_list = [pd.read_json(file) for file in ba_json_files]
ba_ratings = pd.concat(ba_df_list, ignore_index=True)
ba_ratings.head()

C:\Users\athea\AppData\Local\Temp\ipykernel_23104\3299018877.py:3: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  ba_df_list = [pd.read_json(file) for file in ba_json_files]
C:\Users\athea\AppData\Local\Temp\ipykernel_23104\3299018877.py:3: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  ba_df_list = [pd.read_json(file) for file in ba_json_files]


,beer_name,beer_id,brewery_name,brewery_id,style,abv,date,user_name,user_id,appearance,aroma,palate,taste,overall,rating
0,Régab,142544.0,Societe des Brasseries du Gabon (SOBRAGA),37262.0,Euro Pale Lager,4.5,2015-08-20 10:00:00,nmann08,nmann08.184925,3.25,2.75,3.25,2.75,3.0,2.88
1,Barelegs Brew,19590.0,Strangford Lough Brewing Company Ltd,10093.0,English Pale Ale,4.5,2009-02-20 11:00:00,StJamesGate,stjamesgate.163714,3.00,3.50,3.50,4.00,3.5,3.67
2,Barelegs Brew,19590.0,Strangford Lough Brewing Company Ltd,10093.0,English Pale Ale,4.5,2006-03-13 11:00:00,mdagnew,mdagnew.19527,4.00,3.50,3.50,4.00,3.5,3.73
3,Barelegs Brew,19590.0,Strangford Lough Brewing Company Ltd,10093.0,English Pale Ale,4.5,2004-12-01 11:00:00,helloloser12345,helloloser12345.10867,4.00,3.50,4.00,4.00,4.5,3.98
4,Barelegs Brew,19590.0,Strangford Lough Brewing Company Ltd,10093.0,English Pale Ale,4.5,2004-08-30 10:00:00,cypressbob,cypressbob.3708,4.00,4.00,4.00,4.00,4.0,4.00


In [41]:
# Load RateBeer ratings stored in json files into a single DataFrame
rb_json_files = glob.glob(RB_DATA_FOLDER+'*.json')
rb_df_list = [pd.read_json(file) for file in rb_json_files]
rb_ratings = pd.concat(rb_df_list, ignore_index=True)
rb_ratings.head()

C:\Users\athea\AppData\Local\Temp\ipykernel_23104\180671459.py:3: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  rb_df_list = [pd.read_json(file) for file in rb_json_files]
C:\Users\athea\AppData\Local\Temp\ipykernel_23104\180671459.py:3: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  rb_df_list = [pd.read_json(file) for file in rb_json_files]


,beer_name,beer_id,brewery_name,brewery_id,style,abv,date,user_name,user_id,appearance,aroma,palate,taste,overall,rating
0,33 Export (Gabon),410549.0,Sobraga,3198.0,Pale Lager,5.0,2016-04-26 10:00:00,Manslow,175852.0,2.0,4.0,2.0,4.0,8.0,2.0
1,Castel Beer (Gabon),105273.0,Sobraga,3198.0,Pale Lager,5.2,2017-02-17 11:00:00,MAGICuenca91,442761.0,2.0,3.0,2.0,4.0,8.0,1.9
2,Castel Beer (Gabon),105273.0,Sobraga,3198.0,Pale Lager,5.2,2016-06-24 10:00:00,Sibarh,288889.0,3.0,3.0,2.0,3.0,5.0,1.6
3,Castel Beer (Gabon),105273.0,Sobraga,3198.0,Pale Lager,5.2,2016-01-01 11:00:00,fombe89,250510.0,4.0,3.0,1.0,2.0,5.0,1.5
4,Castel Beer (Gabon),105273.0,Sobraga,3198.0,Pale Lager,5.2,2015-10-23 10:00:00,kevnic2008,122778.0,2.0,4.0,2.0,4.0,7.0,1.9


### First look at the data

We will now examine the different DataFrames in more detail.

In [42]:
# explain the columns of users, beers, breweries and ratings DataFrames

**BeerAdvocate beer Dataframe**

In [43]:
ba_beers.sample(4)

,beer_id,beer_name,brewery_id,brewery_name,style,nbr_ratings,nbr_reviews,avg,ba_score,bros_score,abv,avg_computed,zscore,nbr_matched_valid_ratings,avg_matched_valid_ratings
111011,3751,Boston Garden Golden,299,Lowell Beer Works,American Blonde Ale,21,10,3.34,81.0,87.0,4.5,3.330952,NaN,0,NaN
153851,254225,Bramble,30870,La Cabra Brewing,American Porter,2,1,3.89,NaN,NaN,6.3,3.835000,NaN,0,NaN
80740,247530,Shower Beer,38745,Fieldwork Brewing Co.,American Pale Lager,4,1,4.21,NaN,NaN,4.2,4.137500,0.332629,1,4.24
209736,287235,Green Thizz,38798,Idyll Hounds Brewing Company,American Double / Imperial IPA,1,0,4.27,NaN,NaN,8.7,4.270000,0.593297,1,4.27


Let us explain the different columns of the BeerAdvocate beer Dataframe, in which each row is a beer:
- beer_id, beer_name, brewery_id, brewery_name, style are explicit
- nbr_ratings: total number of reviews for that beer, whether or not they include textual reviews
- nbr_reviews: number of reviews for that beer that include textual reviews
- avg: average rating (out of 5) given to the beer based on user ratings
- ba_score: the BeerAdvocate score assigned to the beer, which corresponds to the beer's overall rating within its style category, calculated using a trimmed mean and a custom Bayesian formula that adjusts for the beer's style, balancing the score based on the number of ratings and the style's average
- bros_score: beer rating given by the site’s founders
- abv: 'Alcohol by volume', which indicates the percentage of alcohol content in the beer
- avg_computed: average rating (out of 5) recalculated using a weighted sum of the different aspect ratings
- zscore: z-score of the beer's average rating, which is a statistical measure that indicates how many standard deviations the average rating is from the mean of all ratings from the BeerAdvocate dataset
- nbr_matched_valid_ratings: number of valid ratings for beers that were successfully matched between two BeerAdvocate and RateBeer
- avg_matched_valid_ratings: average rating of those matched and valid ratings across the sites

The last two columns are related to the analysis performed by Robert West and Gael Lederrey in the following paper: https://dlab.epfl.ch/people/west/pub/Lederrey-West_WWW-18.pdf.

**RateBeer beer Dataframe**

In [44]:
rb_beers.sample(4)

,beer_id,beer_name,brewery_id,brewery_name,style,nbr_ratings,overall_score,style_score,avg,abv,avg_computed,zscore,nbr_matched_valid_ratings,avg_matched_valid_ratings
3113,262766,Border Psycho Brutal Imperial Stout,19523,Border Psycho Brewery,Imperial Stout,23,90.0,39.0,3.55,10.3,3.647826,0.352073,23,3.647826
237715,18646,Clay Pipe Backfin Pale Ale,3116,Clay Pipe Brewing Company,American Pale Ale,92,54.0,44.0,3.07,4.8,3.104348,NaN,0,NaN
160523,259305,NOLA Barrel Aged Swamp Grape,10338,NOLA - New Orleans Lager & Ale,Belgian Ale,3,NaN,NaN,3.25,5.0,3.666667,NaN,0,NaN
290870,302941,Ass Clown Jasmine Tea Amber,13421,Ass Clown Brewing Company,Amber Ale,1,NaN,NaN,2.84,6.6,3.400000,NaN,0,NaN


Let us explain the different columns of the RateBeer beer Dataframe, in which each row is a beer:

The beer_id, beer_name, brewery_id, brewery_name, style, nbr_ratings, avg, abv, avg_computed, z-score, nbr_matched_valid_ratings and avg_matched_valid_ratings are the same as for the BeerAdvocate beer Dataframe.

Some columns are missing compared to the BeerAdvocate beer Dataframe: ba_score and bros_score (which makes sense as these are BeerAdvocate-specific scores), and nbr_reviews.

New columns are present compared to the BeerAdvocate beer Dataframe:
- overall_score: score (out of 100) which "reflects the rating given by RateBeer users and how this beer compares to all other beers on RateBeer", calculated by considering the ratings given by each user and the total number of ratings for the beer
- style_score: score given to the beer (out of 100) specifically within its style category

**BeerAdvocate user Dataframe**

In [45]:
ba_users.sample(4)

,nbr_ratings,nbr_reviews,user_id,user_name,joined,location
73441,8,8,hogheadjones.1171637,HogHeadJones,1.472551e+09,"United States, New York"
97463,2,0,ryuheiedo.639557,RyuheiEdo,1.322478e+09,"United States, California"
146224,1,0,mikelinc44.827435,Mikelinc44,1.405591e+09,NaN
66733,5,0,bday.947953,BDay,1.424516e+09,NaN


Let us explain the different columns of the BeerAdvocate user Dataframe, in which each row is a reviewer:
- nbr_ratings, nbr_reviews, user_id, user_name, and location are explicit
- joined: timestamp indicating when the user joined BeerAdvocate in Unix timestamp format (the number of seconds since January 1, 1970, 00:00:00 UTC)

**RateBeer user Dataframe**

In [46]:
rb_users.sample(4)

,nbr_ratings,user_id,user_name,joined,location
9395,455,40749,minch25,1.154772e+09,"United States, Massachusetts"
47752,6,22515,Oregonian,1.116929e+09,"United States, Oregon"
26330,7,358256,guilhermedias,1.423998e+09,Brazil
63154,2,350391,marcosmontesi,1.419764e+09,NaN


Let us explain the different columns of the RateBeer user Dataframe, in which each row is a reviewer:

The columns are the same as in the BeerAdvocate user Dataframe (joined is obviously the timestamp indicating when the user joined RateBeer and not BeerAdvocate), except that nbr_reviews is missing.

**Brewery Dataframes**

In [47]:
ba_breweries.sample(4)

,id,location,name,nbr_beers
10677,410,"United States, Pennsylvania",Pittsburgh Brewing Co.,42
930,25434,England,Buxton Brewery,125
12190,44835,"United States, Vermont",Saint J Brewery,9
12606,37287,"United States, Florida",Zeta Brewing Company,14


In [48]:
rb_breweries.sample(4)

,id,location,name,nbr_beers
21447,4402,England,Goodmanham,2
13223,9123,"United States, Wisconsin",Red Eye Brewing Company,109
9767,32241,Brazil,Dark Side Brewery,1
2548,4949,Lithuania,Davra Alaus Darykla,13


The columns are explicit and are the same for the 2 websites. Each row is a brewery.

**Rating Dataframes**

In [49]:
ba_ratings.sample(4)

,beer_name,beer_id,brewery_name,brewery_id,style,abv,date,user_name,user_id,appearance,aroma,palate,taste,overall,rating
5294750,90 Minute IPA,2093.0,Dogfish Head Brewings & Eats,64.0,American Double / Imperial IPA,9.0,2007-03-28 10:00:00,therica,therica.35737,4.0,3.5,3.5,4.0,3.5,3.73
8198616,4:20 Smuggle Brew,52011.0,Brouwerij Kerkom,3313.0,Belgian Pale Ale,7.5,2009-08-20 10:00:00,wl0307,wl0307.16869,4.5,3.5,4.0,4.0,3.5,3.81
7691683,Yuzu Fierce,186309.0,Off Color Brewing,31678.0,Berliner Weissbier,3.8,2015-08-26 10:00:00,mfnmbvp,mfnmbvp.705321,4.0,4.0,4.5,4.0,4.0,4.05
2696719,Tart Of Darkness,70286.0,The Bruery,16866.0,American Wild Ale,7.0,2013-01-19 11:00:00,Jspriest,jspriest.564915,NaN,NaN,NaN,NaN,NaN,4.75


In [50]:
rb_ratings.sample(4)

,beer_name,beer_id,brewery_name,brewery_id,style,abv,date,user_name,user_id,appearance,aroma,palate,taste,overall,rating
842481,Kvajj Mr. Brown,79582.0,Bryggeriet Kvajj,8574.0,Brown Ale,5.6,2007-12-29 11:00:00,Grovlam,42635.0,2.0,5.0,3.0,6.0,12.0,2.8
1699187,Invercargill Red Hot Chocolate Chili Stout,179331.0,Invercargill Brewery,2268.0,Stout,5.0,2012-07-14 10:00:00,mkel07,82398.0,4.0,7.0,3.0,3.0,17.0,3.4
3352582,Double Take India Pale Ale,146812.0,"World Brews, Winery Exchange Inc.",12165.0,India Pale Ale (IPA),6.9,2013-04-02 10:00:00,bytemesis,170314.0,4.0,4.0,2.0,4.0,8.0,2.2
2509718,Lervig Brewers Reserve Oat IPA,342255.0,Lervig Aktiebryggeri,6163.0,India Pale Ale (IPA),7.2,2016-07-01 10:00:00,weark,125996.0,3.0,8.0,3.0,7.0,14.0,3.5


The columns are the same for the 2 Dataframes. Each row corresponds to an individual review. Most column names are explicit. 
- 'appearance','aroma', 'palate','taste' correspond to aspect ratings (out of 5)
- 'overall' is the mean of the 4 aspect ratings
- 'rating' is the final rating given by the user to the beer

# 0) Data cleaning

In [51]:
# remove useless columns (done)
# make sure each column has the right type (done)
# deal with missing or Nan values
# check the correspondance between brewery_id in the beers DataFrames and brewery_id in the breweries Dataframes
# set all US locations to 'United States' (remove state information)
# remove any embedded HTML links in the location strings
# remove countries with too few reviewers

## Filtering Dataframes

Let us start by removing columns in the different Dataframes that we will not use in our analysis.

The following rows will not be used in our analysis:
nbr_reviews, ba_score, bros_score, abv, avg_computed, zscore, nbr_matched_valid_ratings and avg_matched_valid_ratings, overall_score and style_score.

Let us remove them.

In [52]:
useless_columns_ba = ['nbr_reviews', 'ba_score', 'bros_score', 'abv', 'avg_computed', 'zscore', 'nbr_matched_valid_ratings', 'avg_matched_valid_ratings']
ba_beers = ba_beers.drop(columns=useless_columns_ba)
print(ba_beers.columns)

Index(['beer_id', 'beer_name', 'brewery_id', 'brewery_name', 'style',
       'nbr_ratings', 'avg'],
      dtype='object')


In [53]:
useless_columns_rb = [col for col in useless_columns_ba if col not in ['nbr_reviews','ba_score', 'bros_score']] + ['overall_score', 'style_score']
rb_beers = rb_beers.drop(columns=useless_columns_rb)
print(rb_beers.columns)

Index(['beer_id', 'beer_name', 'brewery_id', 'brewery_name', 'style',
       'nbr_ratings', 'avg'],
      dtype='object')


We will also not use the timestamps indicating the time when users joined the platforms, so let us remove this as well.

In [54]:
ba_users = ba_users.drop(columns='joined')
rb_users = rb_users.drop(columns='joined')
print(ba_users.columns)

Index(['nbr_ratings', 'nbr_reviews', 'user_id', 'user_name', 'location'], dtype='object')


## Verifying value types

Let us verify that the values in the different columns of the different Dataframes have the appropriate type.

In [55]:
print(ba_beers.dtypes,'\n','\n',rb_beers.dtypes)

beer_id           int64
beer_name        object
brewery_id        int64
brewery_name     object
style            object
nbr_ratings       int64
avg             float64
dtype: object 
 
 beer_id           int64
beer_name        object
brewery_id        int64
brewery_name     object
style            object
nbr_ratings       int64
avg             float64
dtype: object


In [56]:
print(ba_users.dtypes,'\n','\n',rb_users.dtypes)

nbr_ratings     int64
nbr_reviews     int64
user_id        object
user_name      object
location       object
dtype: object 
 
 nbr_ratings     int64
user_id         int64
user_name      object
location       object
dtype: object


In [57]:
print(ba_breweries.dtypes,'\n','\n',rb_breweries.dtypes)

id            int64
location     object
name         object
nbr_beers     int64
dtype: object 
 
 id            int64
location     object
name         object
nbr_beers     int64
dtype: object


In [58]:
columns_to_convert = ['beer_name', 'brewery_name', 'style']

ba_beers[columns_to_convert] = ba_beers[columns_to_convert].apply(lambda col: col.astype(str))
rb_beers[columns_to_convert] = rb_beers[columns_to_convert].apply(lambda col: col.astype(str))
print(ba_beers.dtypes,'\n','\n',rb_beers.dtypes)

beer_id           int64
beer_name        object
brewery_id        int64
brewery_name     object
style            object
nbr_ratings       int64
avg             float64
dtype: object 
 
 beer_id           int64
beer_name        object
brewery_id        int64
brewery_name     object
style            object
nbr_ratings       int64
avg             float64
dtype: object


In [59]:
print(ba_ratings.dtypes,'\n','\n',rb_ratings.dtypes)

beer_name               object
beer_id                float64
brewery_name            object
brewery_id             float64
style                   object
abv                    float64
date            datetime64[ns]
user_name               object
user_id                 object
appearance             float64
aroma                  float64
palate                 float64
taste                  float64
overall                float64
rating                 float64
dtype: object 
 
 beer_name               object
beer_id                float64
brewery_name            object
brewery_id             float64
style                   object
abv                    float64
date            datetime64[ns]
user_name               object
user_id                float64
appearance             float64
aroma                  float64
palate                 float64
taste                  float64
overall                float64
rating                 float64
dtype: object


The types of the values in the different columns of the different Dataframes seem appropriate.

## Dealing with missing values

# 1) Link between culture and taste

## a) Beer style preferences

In [60]:
# use clustering techniques to determine beer style is most popular in each country / geographic area
# use time information to determine if regional beer style preferences are stable (which would suggest that they are 
# strongly affected by culture)or if they vary over time

## b) Importance of specific beer attributes

In [61]:
# perform linear regression between attribute ratings the final rating for all countries together and compare coefficients for each attribute
# perform linear regression between attribute ratings the final rating for the different countries separately and observe the distribution of the coefficients for the different attributes across countries

# 2) Location-related biases in ratings

## a) Cultural biases

In [62]:
# determine the final rating for each country/ geographic area
# determine if the final rating for each country/ geographic area is the same using statistical tests

## b) Beer origin bias

In [63]:
# compare the final rating of domestic vs foreign beers and determine if there is a significant difference using statistical tests
# determine if the final rating of a given beer is correlated with the number of reviewers from the country where the beer comes from who reviewed that beer (scatter plot + Pearson’s correlation coefficient + regression)
# isolate beer enthusiasts (who wrote a very large number of reviews) and compare the final rating of domestic vs foreign beers and determine if there is a significant difference using statistical tests

# 3) Other biases

## a) Seasonal biases

In [64]:
# use the time information to determine the season during which each rating was posted (only consider countries with 4 seasons)
# group ratings by season
# within each group, determine the average final rating of each beer style
# compare the results for the different seasons

## b) Experience biais

In [65]:
# isolate users who gave a lot of ratings (based on a chosen threshold)
# for each user, sort their reviews chronologically and assign an "experience level" (predefined values that will be the same for all users: n<o<p) to each rating based the count of reviews posted by that user up to that rating: new reviewer (for the first n reviews), amateur (for the n+1 th review up to the oth review), expert (for the o+1 th review until the last review)
# calculate the average final rating for each experience level across all users
# represent results as a bar plot
# if a particular trend is visible,perform a paired t-test (for early vs. late reviews by the same user) to test if the rating decrease or increase is statistically significant